In [55]:
import pandas as pd
from collections import defaultdict
df = pd.read_csv(r"Real_Data\primaryschool.csv", sep="\t", header=None)

merged = defaultdict(set)

for node_col, attr_col in [(1, 3), (2, 4)]:
    for node, attrs in df.groupby(node_col)[attr_col]:
        merged[node].update(attrs)

meta_df = (
    pd.DataFrame(
        [(node, next(iter(attrs))) for node, attrs in merged.items()],
        columns=["node", "attribute"],
    )
    .sort_values("attribute")
    .reset_index(drop=True)
)

node_mapping = dict(zip(meta_df["node"], meta_df.index))
node_to_attribute = dict(zip(meta_df.index, meta_df["attribute"]))

df = df[[0, 1, 2]]
df.columns = ["starting_times", "source_nodes", "target_nodes"]

df["source_nodes"] = df["source_nodes"].map(node_mapping)
df["target_nodes"] = df["target_nodes"].map(node_mapping)


MIN_TIME = 8.5 * 3600
df["starting_times"] = df["starting_times"] - MIN_TIME
df = df[df["starting_times"] > 24 * 3600].reset_index(drop=True)
df["starting_times"] = df["starting_times"] - 24 * 3600
df['starting_times']=df['starting_times']-df['starting_times'].min()
df["ending_times"] = df["starting_times"] + 20
df['durations']=df["ending_times"] -df["starting_times"] 
df['source_nodes']=df['source_nodes'].astype(int)
df['target_nodes']=df['target_nodes'].astype(int)

df['ending_times']=df['ending_times'].astype(int)
df['starting_times']=df['starting_times'].astype(int)
df['durations']=df['durations'].astype(int)

df['source_nodes']=df['source_nodes'].map(mapping)
df['target_nodes']=df['target_nodes'].map(mapping)

df.to_csv('Pre_processed_Data\primary_school_day2.csv', index=False)

In [56]:
v=set(df['source_nodes']).union(df['target_nodes'])

In [54]:
missing = {121, 172, 207, 213}
all_nodes = list(range(242))  # 1 to 242 inclusive
present_nodes = [n for n in all_nodes if n not in missing]

# Map each present node to a new continuous label starting from 0
mapping = {old: new for new, old in enumerate(present_nodes)}

print(mapping)


{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 122: 121,